# cleaning ast from project file

In [518]:
import json
import pprint
import collections

In [519]:
pp = pprint.PrettyPrinter(indent=4)

In [520]:
project_data = {}
project = 'parsing-test-simple'
with open('data/{}/project.json'.format(project), 'r') as f:
    project_data = json.load(f)

In [521]:
targets = project_data['targets']
sprites = targets[1:]

In [522]:
def get_top_blocks(blocks):
    return {bid: block for bid,block in blocks.items() if 'topLevel' in list(block) and block['topLevel']}

In [523]:
def traverse_blocks(blocks):
    
    """
    traverses the blocks dictionary by continually traversing the 'next' property of each block
    returns a list of top to bottom paths
    """
    
    # 1. Grab the top blocks as reference points
    tops = get_top_blocks(blocks)
    topids = [bid for bid,block in tops.items()]
    
    # Store each stack
    stacks = []
        
    # start at each top block and traverse down by seeing each next block
    for tid in topids:
        stack = traverse_stack([], blocks, tid)
        stacks.append(stack)
    
    return stacks

In [589]:
def traverse_stack(curr_stack, all_blocks, block_id):
    """
    Given a block:
    1. Initialize with its data
    1. Check if it has any children
        2. If it has children,
            recursively traverse each child block
    2. If not, check if it has a next block
    3. Repeat the process with the next block
    """
    
    #Grab the block
    block = all_blocks[block_id]
    
    # Initialize the object to store the current block 
    curr_block = {}
    curr_block['id'] = block_id
    curr_block['type'] = block['opcode']
    curr_block['children'] = []
    curr_block['isLeaf'] = False
    
    # after the keys are added, revert back to regular dict
    curr_block = curr_block
    
    # First check for substacks (control flow)
    for stackid in ['SUBSTACK', 'SUBSTACK2']:
        if stackid in block['inputs']:
            child_id = block['inputs'][stackid][1]
            # pass in the child stack array into the new one
            child_stack = traverse_stack(curr_block['children'], all_blocks, child_id)
            curr_block['children'].append(child_stack)


    # If there's no next block, return the current stack
    # Check if there's a next block
    if block['next'] is None:
        # A lone block is at the top level and has no next 
        if block['topLevel']:
            return [curr_block]
        
        # At the end of a tree
        if len(curr_block['children']) == 0:
            curr_block['isLeaf'] = True
        
        # At a normal end of a branch
        return curr_stack.insert(0,curr_block)

    # Otherwise, go to the next block
    traverse_stack(curr_stack, all_blocks, block['next'])
    
    # prepend because it's recursively added from the bottom up
    curr_stack.insert(0,curr_block)

    return curr_stack

In [590]:
# some utils
def stack_is_flat(stack):
    return all([len(block['children']) == 0 for block in stack])

def stack_is_nested(stack):
    return not stack_is_flat(stack)

In [591]:
all_stacks = traverse_blocks(sprites[0]['blocks'])

In [592]:
def extract(stack):
    """
    Given a stack, this will return all of the paths from
    the top node to its leaves.
    
    # go through all condittionals
    # go from top to bottom
    
    """
    paths = []
    
    # go through the tree and calculate all of the combinations of node -> node paths
    pp.pprint(tree)
    for block in stack:
        path = []
        path.append(block)
        # first go through children:
        if len(block['children']) == 0:
            continue
#         else:
#             for child in children:
                
    
    return paths

In [593]:
def extract_children(child):
    path = []
    if child['isLeaf']:
        return child
    if len(child['children']) == 0:
        extract_children(child)
    return path

In [594]:
# def extract_helper(path):
#     """
#     Given a stack, this will return all of the paths from
#     the top node to its leaves.
#     """
#     if block['isLeaf']:
#         return path
    
#     return extract_helper(path)
#     return 0

In [595]:
# create embeddings first?
"""



"""

'\n\n\n\n'

In [596]:
# maybe one option is simply to look at all of the
# sequential paths down the tree? to every leaf?

In [597]:
for s in all_stacks:
    print("\n")
    pp.pprint(s)
    path = []
    # make a path that goes from every top level to the leaf
#     for block in s:



[   {   'children': [],
        'id': '2U2~47{QSP_bc*?H9{Z/',
        'isLeaf': False,
        'type': 'event_whenflagclicked'},
    {   'children': [   {   'children': [],
                            'id': 'DL+{c_v=TS{^/U)6`p2P',
                            'isLeaf': False,
                            'type': 'motion_goto'},
                        {   'children': [   {   'children': [],
                                                'id': '=cAQ,E,hQ8%*?lIo;8s-',
                                                'isLeaf': True,
                                                'type': 'motion_turnleft'},
                                            None],
                            'id': '~un=l%0OqIOIkj?[}9$}',
                            'isLeaf': False,
                            'type': 'control_if'},
                        <Recursion on list with id=140625150992584>],
        'id': 'W/-6]su)QLx581@kI-?/',
        'isLeaf': False,
        'type': 'control_repeat'},
    {   'childr